In [0]:
# Databricks Notebook: Transform Bronze -> Silver
# 목적: Bronze 테이블(원본 JSON 배열 중심)을 파싱해 정형 컬럼 스키마의 Silver 테이블로 업서트(idempotent)

from pyspark.sql.functions import col, from_json, split
from pyspark.sql.types import ArrayType, StringType
from delta.tables import DeltaTable

# =========================
# (A) 환경/대상 테이블 설정
# =========================
CATALOG = "demo_catalog"
SCHEMA  = "demo_schema"
BRONZE  = f"{CATALOG}.{SCHEMA}.bronze_charts"  # 원본 JSON(row당 kline 배열) 저장
SILVER  = f"{CATALOG}.{SCHEMA}.silver_charts"  # 정형 스키마로 파싱/정제 결과 저장

# =========================
# (B) Silver 테이블 생성(없으면)
#  - Bronze의 raw_json을 파싱해 필요한 컬럼만 보유
#  - dt 파티션으로 읽기/쓰기 범위를 줄여 성능 최적화
# =========================
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {SILVER} (
  symbol       STRING,
  interval     STRING,
  open_time    TIMESTAMP,  -- 캔들 시작 시각(UTC)
  open         DOUBLE,
  high         DOUBLE,
  low          DOUBLE,
  close        DOUBLE,
  volume       DOUBLE,
  unique_key   STRING,     -- "symbol|interval|open_ms"
  event_time   TIMESTAMP,  -- Bronze 이벤트 시각(= open_time와 동일 의미로 사용 가능)
  dt           DATE        -- 파티션 키(= event_time의 날짜)
) USING DELTA
PARTITIONED BY (dt)
""")

# =========================
# (C) Bronze 로드 + 파싱 준비
#  - Binance kline은 JSON 배열 형태이므로 from_json(ArrayType)으로 파싱
#  - unique_key는 "symbol|interval|open_ms" 포맷 → split로 안전 분리
# =========================
bronze = spark.table(BRONZE)

# kline 배열: [0]=open_time(ms), [1]=open, [2]=high, [3]=low, [4]=close, [5]=volume, [6]=close_time(ms), ...
arr = from_json(col("raw_json"), ArrayType(StringType()))

# unique_key 분해: "symbol|interval|open_ms"
uk = split(col("unique_key"), "\\|")

# =========================
# (D) Bronze -> Silver 변환 DF 구성
#  - 필요한 컬럼만 선택하고 타입 캐스팅
#  - dt는 event_time의 날짜를 그대로 사용(파티션 키)
#  - 배치 내부 중복 제거(dropDuplicates)로 idempotent 보장 강화
# =========================
silver_df = (
  bronze
    .withColumn("symbol",     uk.getItem(0))
    .withColumn("interval",   uk.getItem(1))
    .withColumn("open_time",  (arr.getItem(0).cast("long")/1000).cast("timestamp"))
    .withColumn("open",       arr.getItem(1).cast("double"))
    .withColumn("high",       arr.getItem(2).cast("double"))
    .withColumn("low",        arr.getItem(3).cast("double"))
    .withColumn("close",      arr.getItem(4).cast("double"))
    .withColumn("volume",     arr.getItem(5).cast("double"))
    .withColumn("dt",         col("event_time").cast("date"))  # 파티션 키(= 이벤트 날짜)
    .select(
        "symbol","interval","open_time","open","high","low","close","volume",
        "unique_key","event_time","dt"
    )
    .dropDuplicates(["unique_key"])  # 동일 배치 내 중복 방지
)

# =========================
# (E) Delta MERGE (idempotent upsert)
#  - 조인 조건에 dt 포함 → 파티션 프루닝으로 대상 최소화
#  - 동일 unique_key 존재 시 UPDATE, 없으면 INSERT
# =========================
target = DeltaTable.forName(spark, SILVER)

(
  target.alias("t")
    .merge(
      silver_df.alias("s"),
      "t.unique_key = s.unique_key AND t.dt = s.dt"
    )
    .whenMatchedUpdate(set={
        "symbol":     "s.symbol",
        "interval":   "s.interval",
        "open_time":  "s.open_time",
        "open":       "s.open",
        "high":       "s.high",
        "low":        "s.low",
        "close":      "s.close",
        "volume":     "s.volume",
        "event_time": "s.event_time",
        "dt":         "s.dt"
    })
    .whenNotMatchedInsertAll()
    .execute()
)

print(f"Silver transform complete: upserted into {SILVER}")